In [ ]:

from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import os, sys
import copy
import imageio
import scipy.io as io
import random
import skimage
import pandas as pd
import shutil
import pickle

from glob import glob
from random import shuffle
from datetime import datetime
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Activation, BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from sklearn.utils import class_weight
from skimage.transform import rescale, resize, downscale_local_mean
from tqdm import tqdm

import keras.backend as K


fname_glob = glob(r'open\test\*.png')

fname_csv1 = r'open\test_df.csv'

dat_csv1 = pd.read_csv(fname_csv1)

index = dat_csv1.loc[:, "index"]
png = dat_csv1.loc[:, "file_name"]

print(index)

X_img = list()
index_list = list()

for kk, fname1 in tqdm(enumerate(fname_glob)):
    
    if True:
#         print(f'{kk} = {fname1}')
        img = imageio.imread(fname1)
        img2 = resize(img, (256, 256), anti_aliasing=True)

        if img2.ndim==2:
            img2 = np.stack((img2,)*3, axis=-1)
        img2 = 255*img2/np.max(img2)
        img2 = img2.astype(np.uint8)    

        X_img.append(img2)
        index_list.append(index[kk])

kk = 0

plt.figure()
plt.imshow(X_img[kk])
plt.title(f'{index[kk]}')
plt.show()

In [ ]:
X_img = np.stack(X_img, axis=0)
print(X_img.shape)

# 모델 읽어서 예측하기.
gpu_no = '/gpu:0'

##################################
#    예측 1: class 예측하기.

with K.tf.device(gpu_no):
    model1 = keras.models.load_model(r'open\model_check\2022-05-10_101815\CNN_e10.h5', compile=False)
    pscore = model1.predict(X_img, batch_size=1, verbose=0)

print(pscore.shape)
print(pscore[0:10, :])
label_ind_pred = np.argmax(pscore, axis=1)

print(label_ind_pred.shape)
print(label_ind_pred)

class_name = ['bottle', 'cable', 'capsule', 'carpet', 'grid', 'hazelnut', 'leather',
 'metal_nut', 'pill', 'screw', 'tile', 'toothbrush', 'transistor', 'wood', 'zipper']

print(class_name)
label_name_pred = list()
ntest = label_ind_pred.shape[0]
for jj in range(ntest):
    label_name_pred.append(class_name[label_ind_pred[jj]])

# print(label_name_pred)

for kk in range(0, 100, 10):
    plt.figure()
    plt.imshow(X_img[kk,:,:,:])
    plt.title(f'prediction = {label_name_pred[kk]}')
    plt.show()

In [ ]:
##########################################################################
#
#     예측 2: 예측1에서 예측한 class에 해당하는 모델을 읽어서, state 예측하기.
#
#

print(X_img.shape)

with open('state_name_by_class.pickle', 'rb') as f:
    dat = pickle.load(f)

model_dict = dict()
label_list = list()    
    
for jj in tqdm(range(ntest)):
    
    key = label_name_pred[jj]
    state_name = dat[key]

    with K.tf.device(gpu_no):
        if model_dict.get(key) is None:
            model_dict[key] = keras.models.load_model('open/model_check/'+ key + '/CNN_e30.h5', compile=False)
        x = X_img[jj, :,:,:] 
        x1 = x[np.newaxis, :, :, :]
        pscore = model_dict[key].predict(x1, batch_size=1, verbose=0)
        indmax = np.argmax(pscore)

    if jj==100:
        plt.figure()
        plt.imshow(x)
        plt.title(f'{key}-{state_name[0][indmax]}')

    label = f'{key}-{state_name[0][indmax]}'
    label_list.append([jj, label])
 
df = pd.DataFrame (label_list, columns = ['index', 'label'])
        
df.to_csv("sample_submission.csv", header=["index", "label"], index=False)
    